# Práctica 3: Novena prueba

Importamos las librerías necesarias.

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from matplotlib import pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from imblearn.over_sampling import SMOTE

from sklearn.neighbors import LocalOutlierFactor

### Funciones auxiliares

In [2]:
def grafica(data, label, title, xlab, ylab):
    plt.plot(data,label, c='b')
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)

    plt.show()

def graf_barras(data, label, title, xlab, ylab):
    fig, ax = plt.subplots()
    ax.bar(data, label, color='c')
    ax.set_title(title)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)

## Lectura de datos

Leemos los datos:

In [3]:
datos = pd.read_csv("datos/train.csv", na_values=["?"])
test = pd.read_csv("datos/test.csv", na_values=["?"])

year = pd.read_csv("datos/ao.csv", na_values=["?"])
asientos = pd.read_csv("datos/asientos.csv", na_values=["?"])
ciudad = pd.read_csv("datos/ciudad.csv", na_values=["?"])
combustible = pd.read_csv("datos/combustible.csv", na_values=["?"])
consumo = pd.read_csv("datos/consumo.csv", na_values=["?"])
descuento = pd.read_csv("datos/descuento.csv", na_values=["?"])
kilometros = pd.read_csv("datos/kilometros.csv", na_values=["?"])
mano = pd.read_csv("datos/mano.csv", na_values=["?"])
motor_cc = pd.read_csv("datos/motor_cc.csv", na_values=["?"])
nombre = pd.read_csv("datos/nombre.csv", na_values=["?"])
potencia = pd.read_csv("datos/potencia.csv", na_values=["?"])
precio_cat = pd.read_csv("datos/precio_cat.csv", na_values=["?"])
tipo_marchas = pd.read_csv("datos/tipo_marchas.csv", na_values=["?"])

## Preprocesamiento

### Tratamiento de datos perdidos

Eliminamos la variable Descuento, que tiene muchos datos perdidos. Reindexamos y eliminamos la variable recien creada index. Eliminamos el resto de datos con valores perdidos.

In [4]:
p2_datos = datos.copy()
del(p2_datos['Descuento'])
p2_datos = p2_datos.dropna()
p2_datos = p2_datos.reset_index()
del(p2_datos['index'])
p2_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3946 non-null   float64
 1   Nombre        3946 non-null   object 
 2   Ciudad        3946 non-null   object 
 3   Año           3946 non-null   float64
 4   Kilometros    3946 non-null   float64
 5   Combustible   3946 non-null   object 
 6   Tipo_marchas  3946 non-null   object 
 7   Mano          3946 non-null   object 
 8   Consumo       3946 non-null   object 
 9   Motor_CC      3946 non-null   object 
 10  Potencia      3946 non-null   object 
 11  Asientos      3946 non-null   float64
 12  Precio_cat    3946 non-null   int64  
dtypes: float64(4), int64(1), object(8)
memory usage: 400.9+ KB


## Paso de nombres a marcas

In [5]:
for i in range(len(nombre)):
    nombre["Nombre"].iloc[i] = nombre["Nombre"].iloc[i].split(' ')[0]

In [6]:
for i in range(len(p2_datos)):
    p2_datos["Nombre"].iloc[i] = p2_datos["Nombre"].iloc[i].split(' ')[0]

/home/patricia/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Paso de valores categóricas a OneHotVectors

Nombre:

In [7]:
encNombre = OneHotEncoder(handle_unknown='ignore')
encNombre.fit(nombre["Nombre"].to_numpy().reshape(-1, 1))
aux = encNombre.transform(p2_datos["Nombre"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p2_datos["Nombre " + str(i)] = aux[i]
    
del(p2_datos['Nombre'])

Ciudad:

In [8]:
encCiudad = OneHotEncoder(handle_unknown='ignore')
encCiudad.fit(ciudad["Ciudad"].to_numpy().reshape(-1, 1))
aux = encCiudad.transform(p2_datos["Ciudad"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p2_datos["Ciudad " + str(i)] = aux[i]
    
del(p2_datos['Ciudad'])

Combustible:

In [9]:
encCombustible = OneHotEncoder(handle_unknown='ignore')
encCombustible.fit(combustible["Combustible"].to_numpy().reshape(-1, 1))
aux = encCombustible.transform(p2_datos["Combustible"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p2_datos["Combustible " + str(i)] = aux[i]
    
del(p2_datos['Combustible'])

Tipo_marchas:

In [10]:
encTipo_marchas = OneHotEncoder(handle_unknown='ignore')
encTipo_marchas.fit(tipo_marchas["Tipo_marchas"].to_numpy().reshape(-1, 1))
aux = encTipo_marchas.transform(p2_datos["Tipo_marchas"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p2_datos["Tipo_marchas " + str(i)] = aux[i]
    
del(p2_datos['Tipo_marchas'])

Mano:

In [11]:
encMano = OneHotEncoder(handle_unknown='ignore')
encMano.fit(mano["Mano"].to_numpy().reshape(-1, 1))
aux = encMano.transform(p2_datos["Mano"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p2_datos["Mano " + str(i)] = aux[i]
    
del(p2_datos['Mano'])

Consumo:

In [12]:
for i in range(len(p2_datos)):
    p2_datos["Consumo"].iloc[i] = float(p2_datos["Consumo"].iloc[i].strip(' kmlp'))

/home/patricia/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Motor_CC:

In [13]:
for i in range(len(p2_datos)):
    p2_datos["Motor_CC"].iloc[i] = float(p2_datos["Motor_CC"].iloc[i].strip(' CC'))

Potencia:

In [14]:
for i in range(len(p2_datos)):
    p2_datos["Potencia"].iloc[i]= float(p2_datos["Potencia"].iloc[i].strip(' bhp'))

In [15]:
p2_datos["Consumo"] = p2_datos["Consumo"].astype(float)
p2_datos["Motor_CC"] = p2_datos["Motor_CC"].astype(float)
p2_datos["Potencia"] = p2_datos["Potencia"].astype(float)

In [16]:
p2_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              3946 non-null   float64
 1   Año             3946 non-null   float64
 2   Kilometros      3946 non-null   float64
 3   Consumo         3946 non-null   float64
 4   Motor_CC        3946 non-null   float64
 5   Potencia        3946 non-null   float64
 6   Asientos        3946 non-null   float64
 7   Precio_cat      3946 non-null   int64  
 8   Nombre 0        3946 non-null   float64
 9   Nombre 1        3946 non-null   float64
 10  Nombre 2        3946 non-null   float64
 11  Nombre 3        3946 non-null   float64
 12  Nombre 4        3946 non-null   float64
 13  Nombre 5        3946 non-null   float64
 14  Nombre 6        3946 non-null   float64
 15  Nombre 7        3946 non-null   float64
 16  Nombre 8        3946 non-null   float64
 17  Nombre 9        3946 non-null   f

## Separación de objetivo y etiqueta

In [17]:
# Separamos las columnas con los datos de la etiqueta a predecir
cols = [col for col in p2_datos.columns if col not in ['Precio_cat']]
data = p2_datos[cols]
del(data['id'])
target = p2_datos['Precio_cat']

In [18]:
data

,Año,Kilometros,Consumo,Motor_CC,Potencia,Asientos,Nombre 0,Nombre 1,Nombre 2,Nombre 3,...,Combustible 1,Combustible 2,Combustible 3,Combustible 4,Tipo_marchas 0,Tipo_marchas 1,Mano 0,Mano 1,Mano 2,Mano 3
0,2012.0,83000.0,23.40,1248.0,74.00,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,2016.0,4800.0,20.51,998.0,67.04,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,2016.0,26000.0,25.32,1198.0,77.00,6.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,2013.0,56127.0,18.50,1197.0,80.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
4,2017.0,41981.0,18.70,1199.0,88.70,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,2012.0,60000.0,15.20,1968.0,140.80,5.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3942,2012.0,67200.0,15.04,1598.0,103.60,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3943,2014.0,83274.0,22.90,1248.0,74.00,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3944,2015.0,65743.0,20.14,1197.0,81.86,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


## Identificación de outlayers

In [19]:
out_clf = LocalOutlierFactor(n_neighbors=2)
pred = out_clf.fit_predict(data)

index=[]
for i in range(len(pred)):
    if pred[i] == -1:
        index.append(i)
    
print(index)

[3, 4, 6, 10, 14, 16, 19, 27, 30, 38, 39, 41, 43, 44, 45, 46, 47, 51, 53, 55, 58, 62, 67, 79, 80, 86, 101, 102, 103, 106, 107, 112, 114, 116, 130, 132, 134, 136, 142, 143, 156, 158, 165, 170, 173, 179, 182, 183, 194, 198, 199, 201, 205, 207, 208, 216, 218, 219, 221, 223, 226, 228, 231, 232, 234, 242, 244, 245, 251, 256, 263, 267, 274, 277, 279, 283, 293, 294, 297, 299, 301, 308, 309, 310, 311, 313, 318, 330, 343, 347, 348, 351, 354, 357, 361, 365, 370, 385, 386, 388, 397, 398, 407, 410, 411, 412, 416, 418, 424, 434, 438, 446, 447, 448, 449, 452, 458, 461, 463, 465, 466, 467, 480, 482, 484, 486, 487, 491, 492, 496, 497, 498, 506, 507, 511, 516, 517, 520, 522, 523, 527, 530, 536, 537, 539, 541, 543, 544, 547, 550, 557, 563, 569, 575, 579, 582, 583, 587, 596, 597, 601, 605, 610, 611, 617, 624, 627, 630, 633, 638, 648, 649, 657, 660, 673, 675, 679, 680, 681, 682, 683, 684, 685, 687, 689, 690, 695, 700, 717, 718, 719, 720, 723, 725, 726, 732, 738, 741, 743, 747, 749, 751, 753, 774, 780, 782

In [20]:
data = data.drop(index)
target = target.drop(index)
data = data.reset_index()
del(data['index'])
data

,Año,Kilometros,Consumo,Motor_CC,Potencia,Asientos,Nombre 0,Nombre 1,Nombre 2,Nombre 3,...,Combustible 1,Combustible 2,Combustible 3,Combustible 4,Tipo_marchas 0,Tipo_marchas 1,Mano 0,Mano 1,Mano 2,Mano 3
0,2012.0,83000.0,23.40,1248.0,74.00,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,2016.0,4800.0,20.51,998.0,67.04,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,2016.0,26000.0,25.32,1198.0,77.00,6.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,2009.0,140000.0,17.80,1248.0,75.00,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2011.0,60000.0,16.80,1497.0,118.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890,2015.0,73204.0,24.07,998.0,67.10,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2891,2012.0,60000.0,15.20,1968.0,140.80,5.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2892,2014.0,83274.0,22.90,1248.0,74.00,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2893,2015.0,65743.0,20.14,1197.0,81.86,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


## Normalizamos los datos

Normalizamos las variables numéricas.

Año:

In [21]:
scalerYear = MinMaxScaler()
scalerYear.fit(year["Año"].to_numpy().reshape(-1, 1))
aux = scalerYear.transform(data["Año"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
aux
data["Año"] = aux[0]

Kilómetros:

In [22]:
scalerKilometros = MinMaxScaler()
scalerKilometros.fit(kilometros["Kilometros"].to_numpy().reshape(-1, 1))
aux = scalerKilometros.transform(data["Kilometros"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
aux
data["Kilometros"] = aux[0]

Consumo:

In [23]:
for i in range(len(consumo)):
    consumo["Consumo"].iloc[i] = float(consumo["Consumo"].iloc[i].strip(' kmlp'))

scalerConsumo = MinMaxScaler()
scalerConsumo.fit(consumo["Consumo"].to_numpy().reshape(-1, 1))
aux = scalerConsumo.transform(data["Consumo"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
aux
data["Consumo"] = aux[0]

Motor_CC:

In [24]:
for i in range(len(motor_cc)):
    motor_cc["Motor_CC"].iloc[i]= float(motor_cc["Motor_CC"].iloc[i].strip(' CC'))

scalerMotor_CC = MinMaxScaler()
scalerMotor_CC.fit(motor_cc["Motor_CC"].to_numpy().reshape(-1, 1))
aux = scalerMotor_CC.transform(data["Motor_CC"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
aux
data["Motor_CC"] = aux[0]

Potencia:

In [25]:
for i in range(len(potencia)):
    potencia["Potencia"].iloc[i]= float(potencia["Potencia"].iloc[i].strip(' bhp'))

scalerPotencia = MinMaxScaler()
scalerPotencia.fit(potencia["Potencia"].to_numpy().reshape(-1, 1))
aux = scalerPotencia.transform(data["Potencia"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
aux
data["Potencia"] = aux[0]

Asientos:

In [26]:
scalerAsientos = MinMaxScaler()
scalerAsientos.fit(asientos["Asientos"].to_numpy().reshape(-1, 1))
aux = scalerAsientos.transform(data["Asientos"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
aux
data["Asientos"] = aux[0]

In [27]:
data

,Año,Kilometros,Consumo,Motor_CC,Potencia,Asientos,Nombre 0,Nombre 1,Nombre 2,Nombre 3,...,Combustible 1,Combustible 2,Combustible 3,Combustible 4,Tipo_marchas 0,Tipo_marchas 1,Mano 0,Mano 1,Mano 2,Mano 3
0,0.666667,0.012743,0.697674,0.198448,0.075694,0.5,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.857143,0.000712,0.611509,0.156261,0.062457,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.857143,0.003974,0.754919,0.190010,0.081400,0.6,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.523810,0.021513,0.530710,0.198448,0.077596,0.5,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.619048,0.009205,0.500894,0.240466,0.159376,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890,0.809524,0.011236,0.717651,0.156261,0.062571,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2891,0.666667,0.009205,0.453190,0.319946,0.202739,0.5,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2892,0.761905,0.012785,0.682767,0.198448,0.075694,0.5,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2893,0.809524,0.010088,0.600477,0.189841,0.090643,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


## Aplicamos Oversampling

In [28]:
data, target = SMOTE(random_state=15).fit_resample(data, target)

In [29]:
Counter(target)

Counter({3: 1313, 2: 1313, 4: 1313, 5: 1313, 1: 1313})

## Modelo

Modelo final:

In [30]:
# Vectores con los valores de las métricas
gb_clf = GradientBoostingClassifier(random_state=15, n_estimators=760, min_samples_leaf=8, learning_rate=0.1)
mlp_clf = MLPClassifier(random_state=15, hidden_layer_sizes=(300,300),activation='tanh',max_iter=800,early_stopping=True)
rf_clf = RandomForestClassifier(random_state=15, n_estimators=230, criterion = "entropy")

estimators = [('rf', rf_clf), ('mlp', mlp_clf), ('gb', gb_clf)]
st_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(random_state=0, max_iter=400), n_jobs=4)
score = cross_val_score(st_clf, data, target, cv=5)

s = 0
for i in range(len(score)):
    s+= score[i]

print("La accuracy del modelo es: " + str(s/len(score)))

La accuracy del modelo es: 0.9348057882711348


## Predicción del test

Procesamos los datos de test de la misma manera que procesamos los de entrenamiento

In [31]:
p1_test = test.copy()
del(p1_test['Descuento'])
p1_test = p1_test.dropna()
p1_test = p1_test.reset_index()
del(p1_test['index'])
p1_test

,id,Nombre,Ciudad,Año,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos
0,4820,Mercedes-Benz New C-Class C 220 CDI Avantgarde,H,2015,77771,Diesel,Automatic,First,19.27 kmpl,2143 CC,170 bhp,5.0
1,4821,Hyundai i10 Magna 1.1,J,2011,62000,Petrol,Manual,First,19.81 kmpl,1086 CC,68.05 bhp,5.0
2,4822,Ford Figo Diesel EXI,C,2010,129986,Diesel,Manual,Second,20.0 kmpl,1399 CC,68 bhp,5.0
3,4823,Maruti Swift Dzire VXI Optional,D,2015,49900,Petrol,Manual,First,20.85 kmpl,1197 CC,83.14 bhp,5.0
4,4824,Maruti Ritz VXI,J,2010,75319,Petrol,Manual,Second,21.1 kmpl,1197 CC,85.80 bhp,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1154,5974,Chevrolet Beat Diesel LT,G,2012,88000,Diesel,Manual,First,25.44 kmpl,936 CC,57.6 bhp,5.0
1155,5975,Hyundai Verna CRDi,K,2010,72010,Diesel,Manual,First,16.8 kmpl,1493 CC,110 bhp,5.0
1156,5976,Mercedes-Benz A Class A180 CDI,J,2013,34000,Diesel,Automatic,Second,20.0 kmpl,2143 CC,107.3 bhp,5.0
1157,5977,BMW X1 M Sport sDrive 20d,B,2016,31000,Diesel,Automatic,First,17.05 kmpl,1995 CC,190 bhp,5.0


### HotVectors

Nombre:

In [32]:
for i in range(len(p1_test)):
    p1_test["Nombre"].iloc[i] = p1_test["Nombre"].iloc[i].split(' ')[0]

/home/patricia/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [33]:
aux = encNombre.transform(p1_test["Nombre"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p1_test["Nombre " + str(i)] = aux[i]
    
del(p1_test['Nombre'])

Ciudad:

In [34]:
aux = encCiudad.transform(p1_test["Ciudad"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p1_test["Ciudad " + str(i)] = aux[i]
    
del(p1_test['Ciudad'])

Combustible:

In [35]:
aux = encCombustible.transform(p1_test["Combustible"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p1_test["Combustible " + str(i)] = aux[i]
    
del(p1_test['Combustible'])

Tipo_marchas:

In [36]:
aux = encTipo_marchas.transform(p1_test["Tipo_marchas"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p1_test["Tipo_marchas " + str(i)] = aux[i]
    
del(p1_test['Tipo_marchas'])

Mano:

In [37]:
aux = encMano.transform(p1_test["Mano"].to_numpy().reshape(-1, 1)).toarray()
aux = pd.DataFrame(aux)

for i in range(aux.shape[1]):
    p1_test["Mano " + str(i)] = aux[i]
    
del(p1_test['Mano'])

### Pasar a numéricas las variables cuantitativas

Consumo:

In [38]:
for i in range(len(p1_test)):
    p1_test["Consumo"].iloc[i] = float(p1_test["Consumo"].iloc[i].strip(' kmlp'))

/home/patricia/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Motor_CC:

In [39]:
for i in range(len(p1_test)):
    p1_test["Motor_CC"].iloc[i] = float(p1_test["Motor_CC"].iloc[i].strip(' CC'))

Potencia:

In [40]:
for i in range(len(p1_test)):
    p1_test["Potencia"].iloc[i]= float(p1_test["Potencia"].iloc[i].strip(' bhp'))

In [41]:
p1_test["Consumo"] = p1_test["Consumo"].astype(float)
p1_test["Motor_CC"] = p1_test["Motor_CC"].astype(float)
p1_test["Potencia"] = p1_test["Potencia"].astype(float)

### Normalizar:

In [42]:
aux = scalerYear.transform(p1_test["Año"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
p1_test["Año"] = aux[0]

In [43]:
aux = scalerKilometros.transform(p1_test["Kilometros"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
p1_test["Kilometros"] = aux[0]

In [44]:
aux = scalerConsumo.transform(p1_test["Consumo"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
p1_test["Consumo"] = aux[0]

In [45]:
aux = scalerMotor_CC.transform(p1_test["Motor_CC"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
p1_test["Motor_CC"] = aux[0]

In [46]:
aux = scalerPotencia.transform(p1_test["Potencia"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
p1_test["Potencia"] = aux[0]

In [47]:
aux = scalerAsientos.transform(p1_test["Asientos"].to_numpy().reshape(-1, 1))
aux = pd.DataFrame(aux)
p1_test["Asientos"] = aux[0]

### Entrenamos el modelo

In [48]:
ids = p1_test['id']
del(p1_test['id'])

In [49]:
# Vectores con los valores de las métricas
gb_clf = GradientBoostingClassifier(random_state=15, n_estimators=760, min_samples_leaf=8, learning_rate=0.1)
mlp_clf = MLPClassifier(random_state=15, hidden_layer_sizes=(300,300),activation='tanh',max_iter=800,early_stopping=True)
rf_clf = RandomForestClassifier(random_state=15, n_estimators=230, criterion = "entropy")

estimators = [('rf', rf_clf), ('mlp', mlp_clf), ('gb', gb_clf)]
st_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(random_state=0, max_iter=400), n_jobs=4)

st_clf.fit(data, target)
predict = st_clf.predict(p1_test)

In [50]:
df_result = pd.DataFrame({'id': ids, 'Precio_cat': predict})
df_result.to_csv("p3_08.csv", index=False)

In [51]:
predict = pd.read_csv("p3_08.csv", na_values=["?"])

In [52]:
predict

,id,Precio_cat
0,4820,5
1,4821,2
2,4822,2
3,4823,3
4,4824,2
...,...,...
1154,5974,2
1155,5975,3
1156,5976,4
1157,5977,5
